In [1]:
! uv pip install langchain mistralai tiktoken rapidocr-onnxruntime python-dotenv langchain-community

Using Python 3.11.0 environment at: E:\Project\Multi-Doc-Chat\.venv
error: Failed to read metadata for: numpy==2.3.4
  Caused by: failed to open file `E:\Project\Multi-Doc-Chat\.venv\Lib\site-packages\numpy-2.3.4.dist-info\METADATA`: The system cannot find the file specified. (os error 2)


In [2]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["MISTRAL_API_KEY"] = os.getenv("MISTRAL_API_KEY")

## data ingestion

In [3]:
from langchain_community.document_loaders import TextLoader

In [4]:
file_path = "../data/test_data/Agentic_AI_Essay.txt"
loader = TextLoader(file_path)
documents = loader.load()

In [5]:
documents[0].page_content

'\nTitle: The Rise of Agentic AI: A New Paradigm in Artificial Intelligence\n\nIntroduction\nArtificial Intelligence (AI) has evolved dramatically over the last decade, progressing from rule-based expert systems to large-scale neural networks capable of generating language, art, and even scientific hypotheses. However, the next frontier of AI research and development is moving toward something far more dynamic and independentâ€”**Agentic AI**. This new paradigm envisions AI systems that can reason, plan, and act autonomously to achieve complex goals over time. Unlike traditional AI models that passively respond to input, agentic systems can pursue objectives, adapt to environments, and self-improve through feedback loops.\n\nThis essay explores the concept of Agentic AI in depth, covering its definition, architecture, components, current applications, challenges, and potential societal impact. It also discusses the ethical and philosophical dimensions of creating systems capable of aut

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)
text_chunks = text_splitter.split_documents(documents)
text_chunks

[Document(metadata={'source': '../data/test_data/Agentic_AI_Essay.txt'}, page_content='Title: The Rise of Agentic AI: A New Paradigm in Artificial Intelligence'),
 Document(metadata={'source': '../data/test_data/Agentic_AI_Essay.txt'}, page_content='Introduction'),
 Document(metadata={'source': '../data/test_data/Agentic_AI_Essay.txt'}, page_content='Artificial Intelligence (AI) has evolved dramatically over the last decade, progressing from rule-based expert systems to large-scale neural networks capable of generating language, art, and even'),
 Document(metadata={'source': '../data/test_data/Agentic_AI_Essay.txt'}, page_content='art, and even scientific hypotheses. However, the next frontier of AI research and development is moving toward something far more dynamic and independentâ€”**Agentic AI**. This new paradigm'),
 Document(metadata={'source': '../data/test_data/Agentic_AI_Essay.txt'}, page_content='This new paradigm envisions AI systems that can reason, plan, and act autonomous

In [7]:
from langchain_mistralai import MistralAIEmbeddings
from langchain_community.vectorstores import FAISS

In [8]:
embeddings = MistralAIEmbeddings(model="mistral-embed")

e:\Project\Multi-Doc-Chat\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
vectorstore = FAISS.from_documents(text_chunks, embeddings)

In [10]:
vectorstore

In [11]:
retriever = vectorstore.as_retriever()

In [12]:
# Perform similarity search
query = "What is the Key Characteristics of Agentic AI?"
docs = vectorstore.similarity_search(query, k=4)

for i, doc in enumerate(docs):
    print(f"document {i+1}:")
    print(doc.page_content)
    print("-" * 50)

document 1:
1. Defining Agentic AI
--------------------------------------------------
document 2:
The term "agentic" stems from psychology and philosophy, where **agency** describes the capacity of an individual to act independently and make free choices. Translating this to AI implies systems
--------------------------------------------------
document 3:
Agentic AI represents a transformative leap in artificial intelligenceâ€”shifting from reactive systems to proactive, goal-driven entities. By combining perception, reasoning, memory, and
--------------------------------------------------
document 4:
In summary, the rise of Agentic AI marks the dawn of a new eraâ€”one in which intelligence is not merely about cognition, but **about action, adaptation, and responsibility**. With thoughtful design,
--------------------------------------------------


In [13]:
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate


template="""You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use ten sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:
"""

In [14]:
prompt = ChatPromptTemplate.from_template(template)

In [15]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks.\nUse the following pieces of retrieved context to answer the question.\nIf you don't know the answer, just say that you don't know.\nUse ten sentences maximum and keep the answer concise.\nQuestion: {question}\nContext: {context}\nAnswer:\n"), additional_kwargs={})])

In [16]:
from langchain_core.output_parsers import StrOutputParser

In [17]:
output_parser = StrOutputParser()

In [19]:
from langchain_mistralai import ChatMistralAI
llm_model = ChatMistralAI(
    model_name="mistral-small-2506"
)

In [20]:
from langchain_core.runnables import RunnablePassthrough

In [21]:
rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()}
    | prompt
    | llm_model
    | output_parser
)

In [22]:

rag_chain.invoke("tell me about Agentic AI")

'Agentic AI refers to artificial intelligence systems that exhibit **agency**, meaning they can take initiative, make decisions, and perform actions aligned with specific objectives. It represents a shift from reactive systems to proactive, goal-driven entities. These systems combine perception, reasoning, memory, and other capabilities to operate autonomously. Agentic AI is transformative, enabling AI to act independently rather than just respond to inputs. It has applications in various fields, though it also presents challenges and societal impacts. The concept is explored in depth in the provided context, covering its architecture, components, and current uses.'